In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BertForSequenceClassification, BertTokenizer, BertModel

/home/marluxiaboss/anaconda3/envs/llm_detector/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Init Generator and Detector

In [9]:
#GEN_PATH = "microsoft/phi-2"
GEN_PATH = "openai-community/gpt2"
BERT_PATH = "bert-base-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"


class GPTGenerator(nn.Module):
  def __init__(self, gpt_model, tokenizer):
    super().__init__()

    # gpt should already be trained
    self.gpt = gpt_model
    self.tokenizer = tokenizer

  def forward(self, text, max_length=50, temperature=1, top_k=50, top_p=0.9, repetition_penalty=1):
    # tokenize text using the tokenizer
    input_ids = self.tokenizer.encode(text, return_tensors="pt")

    # generate text using the gpt model
    output = self.gpt.generate(input_ids, max_length=max_length, temperature=temperature, top_k=top_k, top_p=top_p)

    # decode the generated text
    decoded_output = self.tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_output

class BertClassifier(nn.Module):
  def __init__(self, bert_model, tokenizer, num_classes):
    super().__init__()

    self.tokenizer = tokenizer

    # bert should already be trained
    self.bert = bert_model

    # set num_classes
    self.num_classes = num_classes

  def forward(self, text):

    # tokenize text using the tokenizer
    output = tokenizer(text, return_tensors="pt")
    input_ids = output["input_ids"]
    logits = self.bert(input_ids)["logits"]

    # apply sigmoid to get probabilities of each class
    output = torch.sigmoid(logits)
    return output
        

In [10]:
torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained(GEN_PATH, torch_dtype="auto", trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(GEN_PATH, trust_remote_code=True)
generator = GPTGenerator(model, tokenizer)

text_input = '''def print_prime(n):
   """
   Print all primes between 1 and n
   """'''

output = generator(text_input)
print(output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def print_prime(n):
   """
   Print all primes between 1 and n
   """    print_prime(n):    """   print_prime(n):   """


In [11]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
detector = BertClassifier(model, tokenizer, 2)

text = "def print_prime(n):\n   \"\"\"\n   Print all primes between 1 and n\n   \"\"\""

logits = detector(text)
fake = logits.argmax().item()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load dataset of instructions and output with gen